In [ ]:
%cd ..

In [1]:
import glob
import os

import pyvista as pv
from natsort import natsorted

In [15]:
def main():
    # Change these values
    data_folder = r'/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01'  # r'/path/to/your/vtkdata/'
    save_folder = r'data/Heart'
    #
    # (glob.glob(data_folder + '*.vtk'))
    print()
    all_data_paths = natsorted(
        [os.path.join(data_folder, dir_name) for dir_name in os.listdir(data_folder) if dir_name.split('.')[-1] == 'vtk']
    )
    # print(glob.glob(data_folder))

    print(all_data_paths)

    for file in all_data_paths:
        case_mesh = pv.get_reader(file).read()
        case_number = file.split(os.sep)[-1].split('.')[0].split('_')[-1]
        filled_case_name = 'full_heart_mesh_' + str(int(case_number)).zfill(3)
        save_path = os.path.join(save_folder, 'heart_seg', 'heart_render', 'heart', filled_case_name)

        if not os.path.isdir(save_path):
            os.makedirs(save_path)

        print(save_path)
        break

        case_mesh.save(os.path.join(save_path, filled_case_name + '.vtk'), binary=True)
        print('Finished file:', file)


main()


['/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_1.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_2.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_3.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_4.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_5.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_6.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_7.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_8.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_Mesh_9.vtk', '/home/suetin/Projects/VSCode/3DReconstructions/Final_models/Final_models_01/Full_Heart_M

In [ ]:
from config import cfg
from src.scripts.data_extraction.slicing.plane_extract import run_slice_extraction
from multiprocessing import Pool, process
from itertools import repeat
import time


def main():
    all_data_paths = natsorted(glob.glob(cfg.DATA_IN.DATA_FOLDER + '*.vtk'))
    start = time.perf_counter()
    case_names = [file.split(os.sep)[-1].split('.')[0] for file in all_data_paths]

    save_paths = [os.path.join(cfg.DATA_IN.SAVE_FOLDER, 'heart_seg', 'heart_render', 'heart', case_name) for case_name
                  in
                  case_names]

    for path in save_paths:
        if not os.path.exists(path):
            os.makedirs(path)

    with Pool(cfg.PARAMETERS.NUM_WORKERS) as pool:
        pool.starmap(run_slice_extraction,
                     zip(repeat(cfg), all_data_paths, save_paths, repeat(cfg.DATA_OUT.FANCY_PLOT)))

    finish = time.perf_counter()
    print(f'Finished in {finish - start} second(s)')


main()

In [ ]:
import numpy as np
import pandas as pd
from natsort import natsorted
from pyntcloud import PyntCloud

from src.scripts.data_extraction.slicing.config import cfg
from src.scripts.data_extraction.slicing.plane_extract import prepare_meshes


def convert_vtk2np_voxels(in_mesh, vol_resolution):
    df = pd.DataFrame(data=in_mesh.points, columns=['x', 'y', 'z'])
    cloud = PyntCloud(df)

    voxelgrid_id = cloud.add_structure('voxelgrid', n_x=vol_resolution, n_y=vol_resolution, n_z=vol_resolution)
    voxelgrid = cloud.structures[voxelgrid_id]
    binary_voxel_array = voxelgrid.get_feature_vector(mode='binary')
    # binary_voxel_array = closing(Binary_voxel_array, cube(2))
    return binary_voxel_array


def main():
    # Change these values
    data_folder = r'/path/to/heart/meshes/'
    save_folder = r'/path/to/save/voxel_models/to'
    vol_resolution = 64
    #

    all_data_paths = natsorted(glob.glob(os.path.join(data_folder, '*.vtk')))

    for file in all_data_paths:
        loaded_mesh, __ = prepare_meshes(cfg, file)
        out_voxel_array = convert_vtk2np_voxels(loaded_mesh, vol_resolution)

        case_name = file.split(os.sep)[-1].split('.')[0]
        save_path = os.path.join(save_folder, 'heart', case_name)

        if not os.path.isdir(save_path):
            os.makedirs(save_path)

        np.save(os.path.join(save_path, 'model.npy'), out_voxel_array)
        print('Finished file:', file)


main()